In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
#Use featureHasher possible .. a good alternative from CountVectorizer, can potentially use less memory  
from sklearn.feature_extraction import FeatureHasher
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
new_data = pd.read_csv('../database-files/output.csv')

In [3]:
new_data

,Unnamed: 0,Source,Dna,zf,f1,f2,f3,ex
0,0,'',ctcgcgGCGgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
1,1,'',ctcgcgGTTgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,RSDERKRHTKI,ex-
2,2,'',ctcgatTACgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,TSGNLVRHTKI,ex-
3,3,'',ctcgatTATgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,TSGNLVRHTKI,ex-
4,4,'',ctcgatTCTgcggcc,3,KSADLKRHIRI,RSDHLTTHIRT,TSGNLVRHTKI,ex-
...,...,...,...,...,...,...,...,...
1165,1165,'',cccgcgGTTgcgtcc,3,KSADLKRHIRI,TADKLSRHIRT,RSDERKRHTKI,ex-
1166,1166,'',cccgcgGCGgcgtcc,3,KSADLKRHIRI,TADKLSRHIRT,RSDERKRHTKI,ex-
1167,1167,'',cccgcgGCAgcgtcc,3,KSADLKRHIRI,TADKLSRHIRT,RSDERKRHTKI,ex-
1168,1168,'',cccgcgGCTgcgtcc,3,KSADLKRHIRI,TADKLSRHIRT,RSDERKRHTKI,ex-


In [4]:
positiveExamples = new_data.loc[(new_data['ex'] == "ex+") | (new_data['ex'] == "+")]
negativeExamples = new_data.loc[(new_data['ex'] == "ex-") | (new_data['ex'] == "-")]

In [5]:
print(positiveExamples)

     Unnamed: 0 Source              Dna  zf           f1           f2  \
8             8     ''  ctcgatAAAgcggcc   3  KSADLKRHIRI  QRANLRAHIRT   
9             9     ''  ctcgatAACgcggcc   3  KSADLKRHIRI  DSGNLRVHIRT   
10           10     ''  ctcgatAAGgcggcc   3  KSADLKRHIRI  RSDTLSNHIRT   
11           11     ''  ctcgatAATgcggcc   3  KSADLKRHIRI  TTGNLTVHIRT   
12           12     ''  ctcgatACAgcggcc   3  KSADLKRHIRI  SPADLTRHIRT   
..          ...    ...              ...  ..          ...          ...   
777         777     ''  cccgcgGGCgcgtcc   3  KSADLKRHIRI  QSSKLVRHIRT   
778         778     ''  cccgcgGGTgcgtcc   3  KSADLKRHIRI  TSGHLVRHIRT   
779         779     ''  cccgcgGGCgcgtcc   3  KSADLKRHIRI  TSGKLVRHIRT   
780         780     ''  cccgcgGGTgcgtcc   3  KSADLKRHIRI  TADHLSRHIRT   
781         781     ''  cccgcgGGGgcgtcc   3  KSADLKRHIRI  TADKLSRHIRT   

              f3   ex  
8    TSGNLVRHTKI  ex+  
9    TSGNLVRHTKI  ex+  
10   TSGNLVRHTKI  ex+  
11   TSGNLVRHTKI  ex+  
12 

In [6]:
print(negativeExamples)

      Unnamed: 0 Source              Dna  zf           f1           f2  \
0              0     ''  ctcgcgGCGgcggcc   3  KSADLKRHIRI  RSDHLTTHIRT   
1              1     ''  ctcgcgGTTgcggcc   3  KSADLKRHIRI  RSDHLTTHIRT   
2              2     ''  ctcgatTACgcggcc   3  KSADLKRHIRI  RSDHLTTHIRT   
3              3     ''  ctcgatTATgcggcc   3  KSADLKRHIRI  RSDHLTTHIRT   
4              4     ''  ctcgatTCTgcggcc   3  KSADLKRHIRI  RSDHLTTHIRT   
...          ...    ...              ...  ..          ...          ...   
1165        1165     ''  cccgcgGTTgcgtcc   3  KSADLKRHIRI  TADKLSRHIRT   
1166        1166     ''  cccgcgGCGgcgtcc   3  KSADLKRHIRI  TADKLSRHIRT   
1167        1167     ''  cccgcgGCAgcgtcc   3  KSADLKRHIRI  TADKLSRHIRT   
1168        1168     ''  cccgcgGCTgcgtcc   3  KSADLKRHIRI  TADKLSRHIRT   
1169        1169     ''  cccgcgGCCgcgtcc   3  KSADLKRHIRI  TADKLSRHIRT   

               f3   ex  
0     RSDERKRHTKI  ex-  
1     RSDERKRHTKI  ex-  
2     TSGNLVRHTKI  ex-  
3     TSGNL

In [7]:
print(len(positiveExamples))
print(len(negativeExamples))

255
915


In [8]:
combined_data = pd.concat([positiveExamples, negativeExamples])

In [9]:
pos_combined_dna_with_zinc = []
pos_dna = []
for index, row in positiveExamples.iterrows():
    pos_seq_dna_with_zinc = (row['Dna'] + row['f1'] + row['f2'] + row['f3'])
    pos_combined_dna_with_zinc.append(pos_seq_dna_with_zinc)
    pos_dna_seq = row['Dna']
    pos_dna.append(pos_dna_seq)

neg_combined_dna_with_zinc = []
neg_dna = []
for index, row in negativeExamples.iterrows():
    neg_seq_dna_with_zinc = (row['Dna'] + row['f1'] + row['f2'] + row['f3'])
    neg_combined_dna_with_zinc.append(neg_seq_dna_with_zinc)
    neg_dna_seq = row['Dna']
    neg_dna.append(neg_dna_seq)

In [10]:
pos_dna

['ctcgatAAAgcggcc',
 'ctcgatAACgcggcc',
 'ctcgatAAGgcggcc',
 'ctcgatAATgcggcc',
 'ctcgatACAgcggcc',
 'ctcgatACCgcggcc',
 'ctcgatACGgcggcc',
 'ctcgatACTgcggcc',
 'ctcgatAGAgcggcc',
 'ctcgatAGGgcggcc',
 'ctcgatAATgcggcc',
 'ctcgatATAgcggcc',
 'ctcgatATGgcggcc',
 'ctcgatATTgcggcc',
 'ctcgatAAGgcggcc',
 'ctcgatAATgcggcc',
 'ctcgatACAgcggcc',
 'ctcgatACAgcggcc',
 'ctcgatACTgcggcc',
 'ctcgatAGGgcggcc',
 'ctcgatATAgcggcc',
 'ctcgatATTgcggcc',
 'ctcgatAAGgcggcc',
 'ctcgatAAGgcggcc',
 'ctcgatAGGgcggcc',
 'ctcgatAGTgcggcc',
 'ctcgatATGgcggcc',
 'ctcgatTAGgcggcc',
 'ctcgatCAAgcggcc',
 'ctcgatCAAgcggcc',
 'ctcgatCAAgcggcc',
 'ctcgatCAAgcggcc',
 'ctcgatCAAgcggcc',
 'ctcgatCCAgcggcc',
 'ctcgatCCAgcggcc',
 'ctcgatCCAgcggcc',
 'ctcgatCCAgcggcc',
 'ctcgatCCAgcggcc',
 'ctcgatCGAgcggcc',
 'ctcgatCGAgcggcc',
 'ctcgatCTAgcggcc',
 'ctcgatCACgcggcc',
 'ctcgatCACgcggcc',
 'ctcgatCACgcggcc',
 'ctcgatCACgcggcc',
 'ctcgatCACgcggcc',
 'ctcgatCCCgcggcc',
 'ctcgatCCCgcggcc',
 'ctcgatCCCgcggcc',
 'ctcgatCGCgcggcc',


In [11]:
neg_dna

['ctcgcgGCGgcggcc',
 'ctcgcgGTTgcggcc',
 'ctcgatTACgcggcc',
 'ctcgatTATgcggcc',
 'ctcgatTCTgcggcc',
 'ctcgatTTAgcggcc',
 'ctcgatTTCgcggcc',
 'ctcgatTTTgcggcc',
 'ctcgatAAGgcggcc',
 'ctcgatACAgcggcc',
 'ctcgatACCgcggcc',
 'ctcgatACGgcggcc',
 'ctcgatACTgcggcc',
 'ctcgatAGAgcggcc',
 'ctcgatAGCgcggcc',
 'ctcgatAGGgcggcc',
 'ctcgatAGTgcggcc',
 'ctcgatATAgcggcc',
 'ctcgatATCgcggcc',
 'ctcgatATGgcggcc',
 'ctcgatATTgcggcc',
 'ctcgatAAAgcggcc',
 'ctcgatAACgcggcc',
 'ctcgatAATgcggcc',
 'ctcgatACAgcggcc',
 'ctcgatACCgcggcc',
 'ctcgatACTgcggcc',
 'ctcgatAGAgcggcc',
 'ctcgatAGCgcggcc',
 'ctcgatAGTgcggcc',
 'ctcgatATAgcggcc',
 'ctcgatATCgcggcc',
 'ctcgatATTgcggcc',
 'ctcgatAAAgcggcc',
 'ctcgatAACgcggcc',
 'ctcgatAAGgcggcc',
 'ctcgatACAgcggcc',
 'ctcgatACCgcggcc',
 'ctcgatACGgcggcc',
 'ctcgatACTgcggcc',
 'ctcgatAGAgcggcc',
 'ctcgatAGCgcggcc',
 'ctcgatAGGgcggcc',
 'ctcgatAGTgcggcc',
 'ctcgatATAgcggcc',
 'ctcgatATCgcggcc',
 'ctcgatATGgcggcc',
 'ctcgatATTgcggcc',
 'ctcgatAAAgcggcc',
 'ctcgatAACgcggcc',


In [12]:
def kmers(sequence, kmer_length=4):
    return[sequence[x:x+kmer_length].lower() for x in range(len(sequence) - kmer_length + 1)]

In [13]:
pos_df = pd.DataFrame(data=pos_combined_dna_with_zinc, columns=['Dna with Zinc'])
neg_df = pd.DataFrame(data=neg_combined_dna_with_zinc, columns=['Dna with Zinc'])

In [14]:
pos_df['kmers_with_zinc'] = pos_df.apply(lambda x: kmers(x['Dna with Zinc']), axis = 1)
#pos_df['Class'] = 0

In [15]:
neg_df['kmers_with_zinc'] = neg_df.apply(lambda x: kmers(x['Dna with Zinc']), axis = 1)
#neg_df['Class'] = 1

In [16]:
pos_df['Dna'] = pos_dna

In [17]:
neg_df['Dna'] = neg_dna

In [18]:
dna_combined = pd.concat([pos_df, neg_df])

In [19]:
pos_and_neg = pd.concat([pos_df, neg_df])
pos_and_neg

,Dna with Zinc,kmers_with_zinc,Dna
0,ctcgatAAAgcggccKSADLKRHIRIQRANLRAHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taaa, aaag, aag...",ctcgatAAAgcggcc
1,ctcgatAACgcggccKSADLKRHIRIDSGNLRVHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taac, aacg, acg...",ctcgatAACgcggcc
2,ctcgatAAGgcggccKSADLKRHIRIRSDTLSNHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taag, aagg, agg...",ctcgatAAGgcggcc
3,ctcgatAATgcggccKSADLKRHIRITTGNLTVHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taat, aatg, atg...",ctcgatAATgcggcc
4,ctcgatACAgcggccKSADLKRHIRISPADLTRHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, atac, taca, acag, cag...",ctcgatACAgcggcc
...,...,...,...
910,cccgcgGTTgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggt, ggtt, gttg, ttg...",cccgcgGTTgcgtcc
911,cccgcgGCGgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggc, ggcg, gcgg, cgg...",cccgcgGCGgcgtcc
912,cccgcgGCAgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggc, ggca, gcag, cag...",cccgcgGCAgcgtcc
913,cccgcgGCTgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggc, ggct, gctg, ctg...",cccgcgGCTgcgtcc


In [20]:
pos_df['kmers_with_dna'] = pos_df.apply(lambda x: kmers(x['Dna']), axis = 1)
neg_df['kmers_with_dna'] = neg_df.apply(lambda x: kmers(x['Dna']), axis = 1)
pos_and_neg = pd.concat([pos_df, neg_df])
pos_and_neg

,Dna with Zinc,kmers_with_zinc,Dna,kmers_with_dna
0,ctcgatAAAgcggccKSADLKRHIRIQRANLRAHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taaa, aaag, aag...",ctcgatAAAgcggcc,"[ctcg, tcga, cgat, gata, ataa, taaa, aaag, aag..."
1,ctcgatAACgcggccKSADLKRHIRIDSGNLRVHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taac, aacg, acg...",ctcgatAACgcggcc,"[ctcg, tcga, cgat, gata, ataa, taac, aacg, acg..."
2,ctcgatAAGgcggccKSADLKRHIRIRSDTLSNHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taag, aagg, agg...",ctcgatAAGgcggcc,"[ctcg, tcga, cgat, gata, ataa, taag, aagg, agg..."
3,ctcgatAATgcggccKSADLKRHIRITTGNLTVHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, ataa, taat, aatg, atg...",ctcgatAATgcggcc,"[ctcg, tcga, cgat, gata, ataa, taat, aatg, atg..."
4,ctcgatACAgcggccKSADLKRHIRISPADLTRHIRTTSGNLVRHTKI,"[ctcg, tcga, cgat, gata, atac, taca, acag, cag...",ctcgatACAgcggcc,"[ctcg, tcga, cgat, gata, atac, taca, acag, cag..."
...,...,...,...,...
910,cccgcgGTTgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggt, ggtt, gttg, ttg...",cccgcgGTTgcgtcc,"[cccg, ccgc, cgcg, gcgg, cggt, ggtt, gttg, ttg..."
911,cccgcgGCGgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggc, ggcg, gcgg, cgg...",cccgcgGCGgcgtcc,"[cccg, ccgc, cgcg, gcgg, cggc, ggcg, gcgg, cgg..."
912,cccgcgGCAgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggc, ggca, gcag, cag...",cccgcgGCAgcgtcc,"[cccg, ccgc, cgcg, gcgg, cggc, ggca, gcag, cag..."
913,cccgcgGCTgcgtccKSADLKRHIRITADKLSRHIRTRSDERKRHTKI,"[cccg, ccgc, cgcg, gcgg, cggc, ggct, gctg, ctg...",cccgcgGCTgcgtcc,"[cccg, ccgc, cgcg, gcgg, cggc, ggct, gctg, ctg..."


In [21]:
pos_df['Class'] = 0
neg_df['Class'] = 1

In [22]:
kmer_with_zinc = list(pos_and_neg['kmers_with_zinc'])
for i in range(len(kmer_with_zinc)):
    kmer_with_zinc[i] = ' '.join(kmer_with_zinc[i])
#class type which would be a positive 0

#convert these using CountVectorizer
#cc = CountVectorizer()
#y_kmer_with_zinc = pos_and_neg.iloc[:,0].values
#y_kmer_with_zinc = cc.fit_transform(y_kmer_with_zinc)

In [23]:
kmer_without_zinc = list(pos_and_neg['kmers_with_dna'])
for i in range(len(kmer_without_zinc)):
    kmer_without_zinc[i] = ' '.join(kmer_without_zinc[i])
#convert these using CountVectorizer
#cc = CountVectorizer()
#y_kmer_without_zinc = (pos_and_neg.iloc[:,2].values)
#y_kmer_without_zinc = cc.fit_transform(y_kmer_without_zinc)

In [24]:
countVectorizer = CountVectorizer(ngram_range=(4,4))
kmer_zinc = countVectorizer.fit_transform(kmer_with_zinc)
kmer_without_zinc = countVectorizer.transform(kmer_without_zinc)

In [25]:
countVectorizer.vocabulary_

{'ctcg tcga cgat gata': 225,
 'tcga cgat gata ataa': 1781,
 'cgat gata ataa taaa': 127,
 'gata ataa taaa aaag': 343,
 'ataa taaa aaag aagc': 57,
 'taaa aaag aagc agcg': 1745,
 'aaag aagc agcg gcgg': 0,
 'aagc agcg gcgg cggc': 2,
 'agcg gcgg cggc ggcc': 25,
 'gcgg cggc ggcc gcck': 398,
 'cggc ggcc gcck ccks': 193,
 'ggcc gcck ccks cksa': 431,
 'gcck ccks cksa ksad': 388,
 'ccks cksa ksad sadl': 122,
 'cksa ksad sadl adlk': 215,
 'ksad sadl adlk dlkr': 1073,
 'sadl adlk dlkr lkrh': 1635,
 'adlk dlkr lkrh krhi': 15,
 'dlkr lkrh krhi rhir': 270,
 'lkrh krhi rhir hiri': 1117,
 'krhi rhir hiri iriq': 1066,
 'rhir hiri iriq riqr': 1351,
 'hiri iriq riqr iqra': 565,
 'iriq riqr iqra qran': 837,
 'riqr iqra qran ranl': 1455,
 'iqra qran ranl anlr': 739,
 'qran ranl anlr nlra': 1265,
 'ranl anlr nlra lrah': 1297,
 'anlr nlra lrah rahi': 52,
 'nlra lrah rahi ahir': 1201,
 'lrah rahi ahir hirt': 1138,
 'rahi ahir hirt irtt': 1294,
 'ahir hirt irtt rtts': 35,
 'hirt irtt rtts ttsg': 617,
 'irtt rtt

In [26]:
#X_train, X_test, y_train, y_test = train_test_split(kmer_zinc, y_kmer_with_zinc)

In [27]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(kmer_without_zinc)
print(y_pred)

NameError: name 'X_train' is not defined

In [ ]:
def metrics(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f" % (accuracy, precision, recall))
    return accuracy, precision, recall

In [ ]:
accuracy, precision, recall = metrics(kmer_without_zinc, y_pred)